In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-dataset/test.csv
/kaggle/input/train-dataset/train.csv
/kaggle/input/test-orig/test.csv
/kaggle/input/submission/sample_submission.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from scipy import stats
import seaborn as sns
from copy import deepcopy

%matplotlib inline


import xgboost as xgb
import pickle
import sys
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix, hstack
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV 

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

## Loading Data
* we load the data.
* Take out categorical and continous feature names.
* Factorize the dataset to make model ready.
* apply log_shift transformation on target feature.

In [3]:
print('Loadind data from csv files.....')
train_data = pd.read_csv('../input/train-dataset/train.csv')
test_data  = pd.read_csv('../input/test-orig/test.csv')
print('shape of train data :',train_data.shape)
print('shape of test data :',test_data.shape)


test_data['loss'] = np.nan
combined_data = pd.concat([train_data, test_data])
print('Joining complete....')
print('shape of combined data:',combined_data.shape)


categorical_features = list(train_data.select_dtypes(include = ['object']).columns)
print('Total number of categorical features in data are :', len(categorical_features))

no_common = []
for i in categorical_features:
    if train_data[i].nunique() != test_data[i].nunique():
        no_common.append(i)
print('Features that have diff unique values in train and test are :')
print(no_common)

def log_xgboost_eval_mae(pred,dtrain):
    labels = dtrain.get_label()
    a = mean_absolute_error(np.exp(pred),np.exp(labels))
    return 'mae',a

def search_feature(x):
    if x in combined_remaining:
        return np.nan
    return 

#Reference https://www.geeksforgeeks.org/python-pandas-factorize/
#Reference https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

for i in categorical_features:
    if train_data[i].nunique() != test_data[i].nunique():
        train_unique_set = set(train_data[i].unique())
        test_unique_set  = set(test_data[i].unique())
        remaining_train  = train_unique_set - test_unique_set
        remaining_test   = test_unique_set - train_unique_set
        
        combined_remaining = remaining_train.union(remaining_test)
        
        combined_data[i] = combined_data[i].apply(lambda x: search_feature(x),1)
    combined_data[i] = pd.factorize(combined_data[i].values,sort = True)[0]
    
x_train = combined_data[combined_data['loss'].notnull()]
x_test = combined_data[combined_data['loss'].isnull()]

shift = 200
y_train = np.log(x_train['loss']+shift)
x_train = x_train.drop(['loss','id'],axis = 1)
x_test  = x_test.drop(['loss','id'],axis = 1)

print(x_train.shape)
print(x_test.shape)

Loadind data from csv files.....
shape of train data : (188318, 132)
shape of test data : (125546, 131)
Joining complete....
shape of combined data: (313864, 132)
Total number of categorical features in data are : 116
Features that have diff unique values in train and test are :
['cat90', 'cat92', 'cat96', 'cat99', 'cat101', 'cat102', 'cat103', 'cat105', 'cat106', 'cat109', 'cat110', 'cat113', 'cat114', 'cat116']
(188318, 130)
(125546, 130)


In [4]:
#Combining both x_train and y_train to do the row sampling 
train = pd.concat([x_train,y_train],axis =1)
total_rows = train.shape[0]
no_of_rows_in_samples = int(total_rows/7)

def row_sampling(train):
    sample_df = train.sample(n = no_of_rows_in_samples,replace = True, random_state=1)
    return sample_df

def split_xy(train):
    x = train.drop(['loss'],axis =1)
    y = train.loss
    return x,y

model_1 = XGBRegressor()
model_2 = AdaBoostRegressor()
model_3 = RandomForestRegressor()
model_4 = LinearRegression()
model_5 = DecisionTreeRegressor()
model_6 = ElasticNet()
model_7 = Lasso()
model_8 = Ridge()

model_list = [model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8]


base_learner_df = pd.DataFrame()
base_learner_output = []
for i in range(len(model_list)):
    train_data1 = row_sampling(train)
    x_train1 = train_data1.drop(['loss'],axis =1)
    y_train1 = train_data1['loss']
    model_list[i].fit(x_train1,y_train1)
    y_pred = model_list[i].predict(x_train)
    base_learner_output.append(y_pred)
    

base_learner_df = pd.DataFrame()
df_cols = ['xgb','ada','rf','lr','dt','eln','las','rid']
for i,j in enumerate(df_cols):
    base_learner_df[j] = base_learner_output[i]

## Explaining above code.
* we concatenated both xtrian and y train cuz we need to do sampling.
* row_sampling function is used to sample the data.
* we did 8 samples of the data and trained 8 different models for each sample respectively.
* used those models to predict whole x train again so that we can use those predictions as input to the meta classifier.
* stored all those predictions into bse_learners_df.
* we will then split that predictions dataframe into x_train and y_trai again to train meta model.

In [5]:
x_train2 = base_learner_df
y_train2 = train['loss']
print(x_train2.shape)
print(y_train2.shape)

(188318, 8)
(188318,)


## Meta classifier as XGBRegressor 

In [6]:
model = XGBRegressor()
model.fit(x_train2,y_train2)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

## Converting Test Data Model ready
* we pass each row on test data on these above trained base models and predict the output.
* we then add all of those predicted output into a dataframe and give that dataframe as a input to meta classifier.


In [11]:
list1=[]
for i in range(0,8):
    list1.append(model_list[0].predict(x_test))
    
test_df = pd.DataFrame()
for i,j in enumerate(df_cols):
    test_df[j] = list1[i]

df = test_df
test_df=(df-df.min())/(df.max()-df.min())

print(test_df.shape)

(125546, 8)


## Observations:
* Test df contains values which we get from base line models. we pass each test row into different base models and save those prediction values into  df.
* and then we pass this df to predict the final output from meta model.
* we then use that predictions as final output.

In [7]:
p_test = np.exp(model.predict(test_df)) - 200
sub = pd.read_csv('../input/submission/sample_submission.csv')
sub['loss'] = p_test
sub.to_csv('submissions7.csv', index=False)

## Observations:
* This model performed decent - we got a score of 1240.46498 MAE value and leader board score is 1109.70.
* we are way worse than normal models.

## Now we will try with standardising the data we get from base line models so that we will feed stardardized input to meta model and see the results.

In [8]:
df = base_learner_df
normalized_df=(df-df.min())/(df.max()-df.min())
x_train2 = normalized_df
y_train2 = train['loss']
print(x_train2.shape)
print(y_train2.shape)

model = XGBRegressor()
model.fit(x_train2,y_train2)

p_test = np.exp(model.predict(test_df)) - 200
sub = pd.read_csv('../input/submission/sample_submission.csv')
sub['loss'] = p_test
sub.to_csv('submissions8.csv', index=False)

(188318, 8)
(188318,)


## Observations:
* This model with standardisation worked really bad - we got a score of 2085.31 and leader board score is 1109.70.
* we are way worse than normal models.

## Final thoughts:
1. why these models worked bad when compared to normal models - one of the main reason is we havent done hyper paramter tuning. Hyper parameter tuning really needed to get best fitting models.
1. usually stacking models will work much better than normal models as we are using multiple models to learn more about data.
1. we can also use KFold techniques to this process to even get better results. But without hyper parameter tuning even kfold will not yield you to get good results.
1. There is a lot of room for improvement here. we got a score of 1114.46 MAE score with good hyper parameter tuned xgboostregressor. so with stacking we can improve our score to like 1112 or 1110 may be, but we need a lot of hyper parameter tuning as we are using so many models.
1. Here we used k samples of data to train on k models. we can always train whole dataset on k models and then we use a meta model on top of that. 
